In [1]:
import numpy as np
import cv2
from imutils import contours

# Load image, grayscale, Gaussian blur, Canny edge detection
image = cv2.imread("test_img/newtest3.jpg")
original = image.copy()

#Convert to grayscale image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Adaptive thresholding
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 4)
# Histogram equalization
equalized = cv2.equalizeHist(thresh)
# Apply Gaussian blur 
blurred = cv2.GaussianBlur(equalized, (3,3), 0)
# Apply canny edge detection
canny = cv2.Canny(blurred, 120, 255, 1)




# # Find contours and extract ROI
# ROI_number = 0
# cnts = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# cnts = cnts[0] if len(cnts) == 2 else cnts[1]
# cnts, _ = contours.sort_contours(cnts, method="left-to-right")
# for c in cnts:
#     x,y,w,h = cv2.boundingRect(c)
#     ROI = original[y:y+h, x:x+w]
#     cv2.rectangle(image, (x,y), (x+w,y+h),(36, 255, 12), 3)
# #     cv2.imwrite('ROI_{}.png'.format(ROI_number), ROI)
#     ROI_number += 1

# print('Contours Detected: {}'.format(ROI_number))

#Display original image
cv2.imshow("image", image) 
# Display grayscale image
cv2.imshow("Grayscale Image", gray)
#Display thresholded image
cv2.imshow("Thresholding image", thresh) 
# Display equalized image
cv2.imshow("Equalized Image", equalized)
# Display blurred image
cv2.imshow("Blurred Image", blurred)
cv2.imshow("canny", canny)
cv2.waitKey()

-1

In [15]:
### Improved Canny Algorithm

import cv2
import numpy as np

def calculate_gradients(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate gradients using 3x3 neighboring area
    Gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    Gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    
    # Compute gradient magnitude and direction
    magnitude = np.sqrt(Gx**2 + Gy**2)
    direction = np.arctan2(Gy, Gx) * (180 / np.pi)
    
    return magnitude, direction

def adaptive_threshold_selection(image, magnitude, mean_percentage=0.15, neighborhood_size=21):
    # Calculate mean gradient magnitude
    mean_magnitude = np.mean(magnitude)
    
    # Initialize output edge map
    edges = np.zeros_like(magnitude)
    
    # Iterate over image pixels
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            # Calculate local neighborhood
            start_row = max(0, i - neighborhood_size // 2)
            end_row = min(image.shape[0], i + neighborhood_size // 2 + 1)
            start_col = max(0, j - neighborhood_size // 2)
            end_col = min(image.shape[1], j + neighborhood_size // 2 + 1)
            local_magnitude = magnitude[start_row:end_row, start_col:end_col]
            
            # Calculate threshold based on local mean and standard deviation
            local_mean = np.mean(local_magnitude)
            local_std = np.std(local_magnitude)
            threshold = local_mean + local_std
            
            # Apply adaptive thresholding
            if magnitude[i, j] >= threshold:
                edges[i, j] = 255
    
    return edges

# Load image
image = cv2.imread('test_img/newtest3.jpg')

# Calculate gradients
magnitude, direction = calculate_gradients(image)

# Adaptive threshold selection
edges = adaptive_threshold_selection(image, magnitude)

# Display results
cv2.imshow('test_img/newtest2.jpg', image)
cv2.imshow('Edge Map', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [7]:
## Traditional Canny Algorithm

import cv2
import numpy as np

def CannyEdgeDetection(image, low_threshold, high_threshold):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to the image
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    
    # Compute gradients using Sobel operator
    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Compute gradient magnitude and direction
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    gradient_direction = np.arctan2(sobel_y, sobel_x) * (180 / np.pi)
    
    # Round gradient direction to four main directions: 0, 45, 90, 135 degrees
    rounded_direction = np.zeros_like(gradient_direction)
    rounded_direction[np.where((gradient_direction >= -22.5) & (gradient_direction < 22.5))] = 0
    rounded_direction[np.where((gradient_direction >= 22.5) & (gradient_direction < 67.5))] = 45
    rounded_direction[np.where((gradient_direction >= -67.5) & (gradient_direction < -22.5))] = 135
    rounded_direction[np.where((gradient_direction >= 67.5) | (gradient_direction < -67.5))] = 90
    
    # Apply non-maximum suppression
    suppressed_magnitude = np.zeros_like(gradient_magnitude)
    for i in range(1, gradient_magnitude.shape[0]-1):
        for j in range(1, gradient_magnitude.shape[1]-1):
            direction = rounded_direction[i, j]
            if direction == 0:
                neighbors = [gradient_magnitude[i, j-1], gradient_magnitude[i, j+1]]
            elif direction == 45:
                neighbors = [gradient_magnitude[i-1, j-1], gradient_magnitude[i+1, j+1]]
            elif direction == 90:
                neighbors = [gradient_magnitude[i-1, j], gradient_magnitude[i+1, j]]
            elif direction == 135:
                neighbors = [gradient_magnitude[i-1, j+1], gradient_magnitude[i+1, j-1]]
            if gradient_magnitude[i, j] >= max(neighbors):
                suppressed_magnitude[i, j] = gradient_magnitude[i, j]
    
    # Apply double thresholding
    high_threshold = np.max(suppressed_magnitude) * high_threshold
    low_threshold = high_threshold * low_threshold
    edges = np.zeros_like(suppressed_magnitude)
    strong_i, strong_j = np.where(suppressed_magnitude >= high_threshold)
    weak_i, weak_j = np.where((suppressed_magnitude >= low_threshold) & (suppressed_magnitude < high_threshold))
    
    edges[strong_i, strong_j] = 255
    edges[weak_i, weak_j] = 50
    
    # Apply edge tracking by hysteresis
    for i in range(1, edges.shape[0]-1):
        for j in range(1, edges.shape[1]-1):
            if edges[i, j] == 50:
                if np.max(edges[i-1:i+2, j-1:j+2]) == 255:
                    edges[i, j] = 255
                else:
                    edges[i, j] = 0
    
    return edges

# Example usage:
image = cv2.imread('test_img/newtest3.jpg')
# edges = CannyEdgeDetection(image, 0.05, 0.15)
edges = CannyEdgeDetection(image, 0.25, 0.3)
# print(image.shape,edges.shape)
cv2.imwrite('canny_edges.jpg', edges)





True

In [2]:
# -*- coding: Game 2048 Digit Recognition -*-

################################################### Best Performance ###########################################
import cv2

img = cv2.imread('test_img/newtest4.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

############################### Histogram equalization ####################################
equalized = cv2.equalizeHist(gray)

cv2.imshow('equalized image',equalized )
cv2.waitKey(0)
cv2.destroyAllWindows()

############################### Binary Thresholding ########################################
ret,thresh = cv2.threshold(equalized,190,255,1)


cv2.imshow('thresh',thresh )
cv2.waitKey(0)
cv2.destroyAllWindows()

############################# Method 1 (Unused)##################################################### 
# # 对二值图像执行膨胀操作
# # kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(2, 2))
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
# # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

# dilated = cv2.dilate(thresh,kernel)

# cv2.imshow('dilated',dilated )
# cv2.waitKey(0)
# cv2.destroyAllWindows()

############################## Method 2 ####################################################
## Perform iterative morphological operations (Opening operation)
# Define the kernel for morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))  # Adjust kernel size as needed

# Perform opening operation (dilation followed by erosion) iteratively
num_iterations = 2  # Number of iterations
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=num_iterations)

# Display the result of opening operation
cv2.imshow('Opened Image', opening)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('temp/opening.jpg', opening)

################################ Extract inside image (Unused) #########################################

# # Find contours in the binary image
# contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the outermost contour (largest contour area)
# outer_contour = max(contours, key=cv2.contourArea)

# # Create a mask for the outer contour
# mask = np.zeros_like(opening)
# cv2.drawContours(mask, [outer_contour], -1, (255), thickness=cv2.FILLED)

# # Apply the mask to the original image to extract the region inside the contour
# image_inside_contour = cv2.bitwise_and(opening, opening, mask=mask)

# # Save or display the extracted region
# cv2.imwrite('image_inside_contour.jpg', image_inside_contour)
# # cv2.imshow('Image Inside Contour', image_inside_contour)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()


############################## Divide into 16 Blocks ###########################################

# Get the dimensions of the original image
height, width = opening.shape

# Calculate the dimensions of each small image
small_image_height = height // 4
small_image_width = width // 4

# Initialize a list to store the small images
small_images = []

# Iterate over the rows of the grid
for i in range(4):
    # Calculate the y-coordinate range for the current row
    y_start = i * small_image_height
    y_end = (i + 1) * small_image_height
    
    # Iterate over the columns of the grid
    for j in range(4):
        # Calculate the x-coordinate range for the current column
        x_start = j * small_image_width
        x_end = (j + 1) * small_image_width
        
        # Extract the small image from the original image
        small_image = opening[y_start:y_end, x_start:x_end]
        
        # Add the small image to the list
        small_images.append(small_image)

# Display or save the small images
for idx, small_image in enumerate(small_images):
    cv2.imwrite(f'temp/small_image_{idx}.jpg', small_image)
    # Uncomment the following line to display the small images
    # cv2.imshow(f'Small Image {idx}', small_image)

# Uncomment the following line to display the small images
# cv2.waitKey(0)
# cv2.destroyAllWindows()

#################### Invert the Binary Image #########################################
# # Invert the colors of each small image
# inverted_images = [255 - image for image in small_images]

# # Display or save the inverted images
# for idx, inverted_image in enumerate(inverted_images):
#     cv2.imwrite(f'inverted_image_{idx}.jpg', inverted_image)
#     # Uncomment the following line to display the inverted images
#     # cv2.imshow(f'Inverted Image {idx}', inverted_image)

# # Uncomment the following line to display the inverted images
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()

####################### Resize to the Standard 187 * 186 Image #################################################
# Define the desired dimensions for resizing
new_width = 187
new_height = 186


# Resize each small image
resized_images = [cv2.resize(image, (new_width, new_height)) for image in small_images]

## Perform iterative morphological operations for each resized image

# Define the kernel for morphological operations
kernel_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))  # Adjust kernel size as needed
# Perform opening operation (dilation followed by erosion) iteratively
num_iterations = 2  # Number of iterations
    
for img in resized_images:
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel_1, iterations=num_iterations)

# Display or save the resized images
for idx, resized_image in enumerate(resized_images):
    cv2.imwrite(f'input_img/resized_image_{idx}.jpg', resized_image)
    # Uncomment the following line to display the resized images
    # cv2.imshow(f'Resized Image {idx}', resized_image)
    


In [142]:
############################## Digit Recognition #################################

import cv2
import os

# Define the folder containing input images
input_folder = 'input_img/'

# Define the folder containing the standard labeled digit images
standard_digit_folder = 'label_img/'

# Define the threshold for the correlation score
correlation_threshold = 0.3  # Adjust as neede

# Initialize a 4x4 matrix to store recognized digits
recognized_matrix = [['' for _ in range(4)] for _ in range(4)]

# Iterate through each input image
for input_filename in os.listdir(input_folder):
    # Load the input image containing a digit
    input_image = cv2.imread(os.path.join(input_folder, input_filename), cv2.IMREAD_GRAYSCALE)
    
    # Initialize variables to store the maximum correlation score and recognized digit
    max_correlation_score = -1
    recognized_digit = None
    
    # Iterate through each standard digit image
    for digit_filename in os.listdir(standard_digit_folder):
        # Load the standard digit image
        standard_digit_image = cv2.imread(os.path.join(standard_digit_folder, digit_filename), cv2.IMREAD_GRAYSCALE)
        
        # Calculate the correlation score between the input image and the standard digit image
        correlation_score = cv2.matchTemplate(input_image, standard_digit_image, cv2.TM_CCOEFF_NORMED)[0][0]
        
        # Update the recognized digit if the correlation score is higher
        if correlation_score > max_correlation_score:
            max_correlation_score = correlation_score
            recognized_digit = digit_filename.split(".")[0].split("_")[-1]  # Extract the digit from the filename
    
    # Check if the correlation score exceeds the threshold
    if max_correlation_score >= correlation_threshold:
        # Output the recognized digit for the current input image
        
        # Extract the row and column indices from the input filename
        
        index = int(input_filename.split('.')[0].split('_')[-1])
        print(index)
        row,col= index//4,index % 4
        
        # Update the recognized matrix with the recognized digit
        recognized_matrix[row][col] = int(recognized_digit)
        
        print(max_correlation_score)
        print(f"Recognized Digit for {input_filename}: {recognized_digit}")
    else:
        # Output that the input image does not contain any digit
        print(max_correlation_score)
        print(f"No digit recognized for {input_filename}")
##########################  Output Matrix ####################################
print(np.array(recognized_matrix))

0.0
No digit recognized for resized_image_0.jpg
1
0.53949696
Recognized Digit for resized_image_1.jpg: 2
10
0.34915483
Recognized Digit for resized_image_10.jpg: 128
11
0.36490333
Recognized Digit for resized_image_11.jpg: 4
0.0
No digit recognized for resized_image_12.jpg
0.062492035
No digit recognized for resized_image_13.jpg
14
0.4537652
Recognized Digit for resized_image_14.jpg: 2
0.24829598
No digit recognized for resized_image_15.jpg
2
0.5611999
Recognized Digit for resized_image_2.jpg: 16
0.18146123
No digit recognized for resized_image_3.jpg
4
0.38060892
Recognized Digit for resized_image_4.jpg: 2
5
0.37042758
Recognized Digit for resized_image_5.jpg: 16
6
0.5880812
Recognized Digit for resized_image_6.jpg: 16
7
0.38267523
Recognized Digit for resized_image_7.jpg: 4
0.14240648
No digit recognized for resized_image_8.jpg
9
0.647941
Recognized Digit for resized_image_9.jpg: 2
[['' '2' '16' '']
 ['2' '16' '16' '4']
 ['' '2' '128' '4']
 ['' '' '2' '']]


In [131]:
########################### Build the Standard Library #########################################
img = cv2.imread('label_img/black_2048.jpg')
# Invert the colors of each small image
inverted_image = 255 - img
cv2.imwrite("label_img/new_white_2048.jpg", inverted_image)

True

In [114]:
##############################   Revised version #################################
######## We divide the original image into 16 blocks first and then  perform the following operations #########

# -*- coding: Game 2048 Digit Recognition -*-

################################################### Best Performance ###########################################
import cv2

img = cv2.imread('test_img/newtest4.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

############################## Divide into 16 Blocks ###########################################

# Get the dimensions of the original image
height, width = gray.shape

# Calculate the dimensions of each small image
small_image_height = height // 4
small_image_width = width // 4

# Initialize a list to store the small images
small_images = []

# Iterate over the rows of the grid
for i in range(4):
    # Calculate the y-coordinate range for the current row
    y_start = i * small_image_height
    y_end = (i + 1) * small_image_height
    
    # Iterate over the columns of the grid
    for j in range(4):
        # Calculate the x-coordinate range for the current column
        x_start = j * small_image_width
        x_end = (j + 1) * small_image_width
        
        # Extract the small image from the original image
        small_image = opening[y_start:y_end, x_start:x_end]
        
        # Add the small image to the list
        small_images.append(small_image)

# Display or save the small images
for idx, small_image in enumerate(small_images):
    cv2.imwrite(f'temp/small_image_{idx}.jpg', small_image)
    # Uncomment the following line to display the small images
    # cv2.imshow(f'Small Image {idx}', small_image)

# Uncomment the following line to display the small images
# cv2.waitKey(0)
# cv2.destroyAllWindows()

####################### Resize to the Standard 187 * 186 Image #################################################
# Define the desired dimensions for resizing
new_width = 187
new_height = 186


# Resize each small image
for img in small_images:
    img = cv2.resize(img, (new_width, new_height))

############################### Histogram equalization ####################################
for img in small_images:
    img = cv2.equalizeHist(img)
    

# cv2.imshow('equalized image',equalized )
# cv2.waitKey(0)
# cv2.destroyAllWindows()

############################### Binary Thresholding ########################################
for img in small_images:
    ret,thresh = cv2.threshold(equalized,190,255,1)
    img = ret

# cv2.imshow('thresh',thresh )
# cv2.waitKey(0)
# cv2.destroyAllWindows()

############################# Method 1 ##################################################### 
# # 对二值图像执行膨胀操作
# # kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(2, 2))
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
# # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

# dilated = cv2.dilate(thresh,kernel)

# cv2.imshow('dilated',dilated )
# cv2.waitKey(0)
# cv2.destroyAllWindows()

############################## Method 2 ####################################################
## Perform iterative morphological operations (Opening operation)
# Define the kernel for morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))  # Adjust kernel size as needed

# Perform opening operation (dilation followed by erosion) iteratively
num_iterations = 2  # Number of iterations
for img in small_images:
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=num_iterations)

# Display the result of opening operation
# cv2.imshow('Opened Image', opening)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# cv2.imwrite('temp/opening.jpg', opening)

################################ Extract inside image #########################################

# # Find contours in the binary image
# contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Find the outermost contour (largest contour area)
# outer_contour = max(contours, key=cv2.contourArea)

# # Create a mask for the outer contour
# mask = np.zeros_like(opening)
# cv2.drawContours(mask, [outer_contour], -1, (255), thickness=cv2.FILLED)

# # Apply the mask to the original image to extract the region inside the contour
# image_inside_contour = cv2.bitwise_and(opening, opening, mask=mask)

# # Save or display the extracted region
# cv2.imwrite('image_inside_contour.jpg', image_inside_contour)
# # cv2.imshow('Image Inside Contour', image_inside_contour)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()




#################### Invert the Binary Image #########################################
# # Invert the colors of each small image
# inverted_images = [255 - image for image in small_images]

# # Display or save the inverted images
# for idx, inverted_image in enumerate(inverted_images):
#     cv2.imwrite(f'inverted_image_{idx}.jpg', inverted_image)
#     # Uncomment the following line to display the inverted images
#     # cv2.imshow(f'Inverted Image {idx}', inverted_image)

# # Uncomment the following line to display the inverted images
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()


# Display or save the resized images
for idx, small_image in enumerate(small_images):
    cv2.imwrite(f'input_img/resized_image_{idx}.jpg', small_image)
    # Uncomment the following line to display the resized images
    # cv2.imshow(f'Resized Image {idx}', resized_image)
    
